In [8]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

In [9]:
monthly_noise = pd.read_csv('/Users/tianying/Documents/Statistics and Data Science/Course/Modern Data Analytics/MDA project/MDA-Georgia/Data/monthly_noisedata_2022.csv', header = 0, sep=',')
daily_noise = pd.read_csv('/Users/tianying/Documents/Statistics and Data Science/Course/Modern Data Analytics/MDA project/MDA-Georgia/Data/daily_noisedata_2022.csv', header = 0, sep=',')
hourly_noise = pd.read_csv('/Users/tianying/Documents/Statistics and Data Science/Course/Modern Data Analytics/MDA project/MDA-Georgia/Data/hourly_noisedata_2022.csv', header = 0, sep=',')

In [10]:
# create a dataframe
gps_data = {
    'description': ['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears', 'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81', 'MP08bis - Vrijthof'],
    'lat': [50.877121, 50.87650, 50.87590, 50.875237, 50.87452, 50.874078, 50.873808, 50.87873],
    'lon': [4.700708, 4.700632, 4.700262, 4.700071, 4.69985, 4.70005, 4.700007, 4.70115]
}

gps_df = pd.DataFrame(gps_data)
gps_df.head(10)
  

,description,lat,lon
0,MP 01: Naamsestraat 35 Maxim,50.877121,4.700708
1,MP 02: Naamsestraat 57 Xior,50.876500,4.700632
2,MP 03: Naamsestraat 62 Taste,50.875900,4.700262
3,MP 04: His & Hears,50.875237,4.700071
4,MP 05: Calvariekapel KU Leuven,50.874520,4.699850
5,MP 06: Parkstraat 2 La Filosovia,50.874078,4.700050
6,MP 07: Naamsestraat 81,50.873808,4.700007
7,MP08bis - Vrijthof,50.878730,4.701150


In [17]:
## Monthly noise
# merge noise data with gps
merged_monthly = pd.merge(monthly_noise, gps_df, on='description', how='left')

# Create a StandardScaler object
scaler = MinMaxScaler()

# Fit the StandardScaler to the column and transform the lamax values
merged_monthly['standardized_lamax'] = scaler.fit_transform(merged_monthly[['lamax']])

# Fit the StandardScaler to the column and transform the lamax values
merged_monthly['standardized_laeq'] = scaler.fit_transform(merged_monthly[['laeq']])

merged_monthly.head()

,month,description,lamax,laeq,year,standardized_lamax,standardized_laeq,lat,lon
0,1,MP 03: Naamsestraat 62 Taste,53.239009,51.727544,2022,0.656721,0.688397,50.875900,4.700262
1,1,MP 05: Calvariekapel KU Leuven,50.374792,48.560197,2022,0.417642,0.407849,50.874520,4.699850
2,1,MP 06: Parkstraat 2 La Filosovia,50.086348,48.274795,2022,0.393565,0.382569,50.874078,4.700050
3,1,MP 07: Naamsestraat 81,48.800443,47.515371,2022,0.286230,0.315303,50.873808,4.700007
4,2,MP 01: Naamsestraat 35 Maxim,57.287668,55.245502,2022,0.994666,1.000000,50.877121,4.700708


In [12]:
## Daily noise
# merge noise data with gps
merged_daily = pd.merge(daily_noise, gps_df, on='description', how='left')
# Add a new column 'year' with value 2022 for all observations
merged_daily['year'] = 2022

# Create a new 'date' column by combining 'year', 'month', and 'day'
merged_daily['date'] = pd.to_datetime(merged_daily[['year', 'month', 'day']])

reference_date = pd.to_datetime('2022-01-01')
merged_daily['days_since_reference'] = (merged_daily['date'] - reference_date).dt.days

# Create a StandardScaler object
scaler = MinMaxScaler()

# Fit the StandardScaler to the column and transform the lamax values
merged_daily['standardized_lamax'] = scaler.fit_transform(merged_daily[['lamax']])

# Fit the StandardScaler to the column and transform the lamax values
merged_daily['standardized_laeq'] = scaler.fit_transform(merged_daily[['laeq']])

merged_daily.head()

,month,day,description,lamax,laeq,lat,lon,year,date,days_since_reference,standardized_lamax,standardized_laeq
0,1,1,MP 03: Naamsestraat 62 Taste,51.665242,49.992637,50.875900,4.700262,2022,2022-01-01,0,0.515869,0.538979
1,1,1,MP 05: Calvariekapel KU Leuven,48.747476,46.504067,50.874520,4.699850,2022,2022-01-01,0,0.415931,0.416866
2,1,1,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220,50.874078,4.700050,2022,2022-01-01,0,0.399577,0.399474
3,1,1,MP 07: Naamsestraat 81,45.908501,44.373056,50.873808,4.700007,2022,2022-01-01,0,0.318692,0.342272
4,1,2,MP 03: Naamsestraat 62 Taste,51.407297,50.094018,50.875900,4.700262,2022,2022-01-02,1,0.507034,0.542528


In [13]:
# check for missing values in each column
print(merged_daily.isnull().sum())

month                   0
day                     0
description             0
lamax                   0
laeq                    0
lat                     0
lon                     0
year                    0
date                    0
days_since_reference    0
standardized_lamax      0
standardized_laeq       0
dtype: int64


In [19]:
## Monthly noise mao
# Add initial trace to the figure
fig_monthly = px.scatter_mapbox(merged_monthly, 
                        lat = 'lat', 
                        lon = 'lon', 
                        size = 'standardized_lamax',
                        size_max = 30,
                        animation_frame="month",
                        hover_data={"description":True,"lat":False,"lon":False,"standardized_lamax":False,"lamax":True},
                        zoom = 14, mapbox_style = 'open-street-map')
# Set the initial center and zoom level of the map
fig_monthly.update_layout(mapbox={
    'center': {'lat': 50.875900, 'lon': 4.700262}
})

In [21]:
## Daily noise map
# Add initial trace to the figure
fig = px.scatter_mapbox(merged_daily, 
                        lat = 'lat', 
                        lon = 'lon', 
                        size = 'standardized_lamax',
                        size_max = 30,
                        animation_frame="days_since_reference",
                        hover_data={"description":True,"days_since_reference":False,"lat":False,"lon":False,"date": True,"standardized_lamax":False,"lamax":True},
                        zoom = 14, mapbox_style = 'open-street-map')
# Set the initial center and zoom level of the map
fig.update_layout(mapbox={
    'center': {'lat': 50.875900, 'lon': 4.700262}
})

In [ ]:
def marker_click(trace, points, state):
    # Get the clicked marker information
    lat = trace.lat[points.point_inds[0]]
    lon = trace.lon[points.point_inds[0]]
    size = trace.marker.size[points.point_inds[0]]
    text = trace.text[points.point_inds[0]]

    # Perform actions based on the clicked marker
    # For example, display a pop-up or generate a new visualization

# Assign the callback function to the scattermapbox trace
fig.data[0].on_click(marker_click)

# Add dropdown for switching between Lamax and Laeq
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"marker.size": merged['standardized_lamax']}],
                    label="Lamax",
                    method="restyle"
                ),
                dict(
                    args=[{"marker.size": merged['standardized_laeq']}],
                    label="Laeq",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.3,
            yanchor="top"
        ),
    ]
)

# Show the map
fig.show()